## NLP处理实例

### 数据集: Disasters on social media

推特（社交媒体）上有许多的信息，其中有关于灾难，疾病，暴乱的，有些只是开玩笑或者是电影情节，我们该如何让机器能分辨出这两种讨论呢？

### 为什么这很重要
我们将努力正确预测有关灾难的推文。这是一个非常相关的问题，因为：

* 对于任何试图从噪音中获取信号的人来说，这都是可以采取行动的(比如警察部门在这种情况下)。
* 这是很棘手的，因为依赖关键字比在大多数情况下像垃圾邮件更难。

###  任务
对文本数据进行分类，并分析找到相关归因。

In [19]:
import sklearn
from tensorflow import keras
import nltk
import pandas as pd
import numpy as np
import re
import codecs

### 数据输入
需要对数据做一下编码处理

In [25]:
input_file = codecs.open("data/socialmedia_relevant_cols.csv", "r",encoding='utf-8', errors='replace')
output_file = open("data/socialmedia_relevant_cols_clean.csv", "w",encoding='utf-8')

def sanitize_characters(raw, clean):    
    for line in input_file:
        out = line
        output_file.write(line)

In [26]:
sanitize_characters(input_file, output_file)

### 数据预处理

In [28]:
questions = pd.read_csv("data/socialmedia_relevant_cols_clean.csv")
questions.columns=['text', 'choose_one', 'class_label']
questions.head()

,text,choose_one,class_label
0,Just happened a terrible car crash,Relevant,1
1,Our Deeds are the Reason of this #earthquake M...,Relevant,1
2,"Heard about #earthquake is different cities, s...",Relevant,1
3,"there is a forest fire at spot pond, geese are...",Relevant,1
4,Forest fire near La Ronge Sask. Canada,Relevant,1
